## edgeR pipeline followed for filtering genes and normalising gene-expression
1. Unnormalised and Normalised gene expression boxplots 
2. MDS Plots
3. Gene/Protein expression heatmaps 

## Spatial Deconvolution
1. Reference matrix heatmap
2. Cell type deconvolution barplot
3. Cell type proportion difference in PanCK vs Stroma boxplot 

# Loading Libraries

In [ ]:
#################################################################################################################################################
# Load required libraries
#################################################################################################################################################

library(readxl)
library(patchwork)
library(reshape2)
library(SpatialDecon)
library(clusterSim)
library(edgeR)
library(limma)
library(Glimma)
library(org.Mm.eg.db)
library(tidyverse)
library(dplyr)
library(stringr)
library(reticulate)
library(janitor)
library(tibble)
library(UpSetR)
library(factoextra)
library(gplots)
library(ggplot2)
library(ggpubr)
library(RColorBrewer)
library(pheatmap)
options(warn=-1)

# Loading Data

In [ ]:
#################################################################################################################################################
# Load Data and Metadata
#################################################################################################################################################

#Path = "/QRISdata/Q2051/NanoString_SCC/"
Path = "D:/Onkar_D/UQ/Project_NanoString Single Cell/NanoString_SCC/csvFiles/"

data("safeTME")
data("safeTME.matches")
data("cellcols")

all_cancer_PatientOverlapResults <- read.delim(paste0(Path,"all_cancer_PatientOverlapResults.txt"))
all_normal_PatientOverlapResults <- read.delim(paste0(Path,"all_normal_PatientOverlapResults.txt"))
CTA_Bioprobe_QC <- read_excel(paste0(Path,"All Data CTA.xlsx"), sheet = "Q3")
Protein_QC <- read_excel(paste0(Path,"All Data Human IO Protein.xlsx"), sheet = "HouseKeeping Normalized")
Segment_Properties <- read.csv(paste0(Path,"Segment_Properties.csv"))
Segment_Properties <- Segment_Properties[seq_len(nrow(Segment_Properties)) + c(1,-1),]


#################################################################################################################################################
# RNA Data Wrangling
#################################################################################################################################################

CTA_Bioprobe_QC$ScanLabel <- substring(CTA_Bioprobe_QC$ScanLabel, 5)
CTA_Bioprobe_QC$SegmentLabel <- paste(CTA_Bioprobe_QC$SegmentLabel, CTA_Bioprobe_QC$ROILabel, CTA_Bioprobe_QC$ScanLabel)
CTA_Bioprobe_QC_SC <- CTA_Bioprobe_QC[,-c(1:2)] 
CTA_Bioprobe_QC_SC$SegmentLabel <- gsub(" ", "_", CTA_Bioprobe_QC_SC$SegmentLabel)
CTA_Bioprobe_QC_SC <- as.data.frame(CTA_Bioprobe_QC_SC)
CTA_Bioprobe_QC_SC <- data.frame(CTA_Bioprobe_QC_SC[,-1], row.names = CTA_Bioprobe_QC_SC[,1])
row.names(CTA_Bioprobe_QC_SC)[1:12] <- substring(row.names(CTA_Bioprobe_QC_SC)[1:12],1,12)
row.names(CTA_Bioprobe_QC_SC)[13:18] <- substring(row.names(CTA_Bioprobe_QC_SC)[13:18],1,12)
row.names(CTA_Bioprobe_QC_SC)[19:22] <- paste0(substring(row.names(CTA_Bioprobe_QC_SC)[19:22],1,9),"P04")
row.names(CTA_Bioprobe_QC_SC)[23:24] <- substring(row.names(CTA_Bioprobe_QC_SC)[23:24],1,12)
row.names(CTA_Bioprobe_QC_SC) <- gsub("Stroma","CD45+",rownames(CTA_Bioprobe_QC_SC))
CTA_Bioprobe_QC_SC <- CTA_Bioprobe_QC_SC[order(row.names(CTA_Bioprobe_QC_SC)),]
row.names(CTA_Bioprobe_QC_SC)[1:12] <- paste0(substr(row.names(CTA_Bioprobe_QC_SC)[1:12],1,5),substr(row.names(CTA_Bioprobe_QC_SC)[1:12],8,11),
                                             substr(row.names(CTA_Bioprobe_QC_SC)[1:12],6,7))
row.names(CTA_Bioprobe_QC_SC)[13:24] <- paste0(substr(row.names(CTA_Bioprobe_QC_SC)[13:24],1,6),substr(row.names(CTA_Bioprobe_QC_SC)[13:24],9,12),
                                             substr(row.names(CTA_Bioprobe_QC_SC)[13:24],7,8))
CTA_Bioprobe_QC_SC <- CTA_Bioprobe_QC_SC[order(row.names(CTA_Bioprobe_QC_SC),decreasing=T),]

#################################################################################################################################################
# Protein Data Wrangling
#################################################################################################################################################

Protein_QC$`Segment tags` <- paste(Protein_QC$`Segment tags`, Protein_QC$ROI_ID, Protein_QC$Scan_ID)
Protein_QC_SC <- Protein_QC[,-c(1:6,8,9)] 
Protein_QC_SC$`Segment tags` <- gsub(" ", "_", Protein_QC_SC$`Segment tags`)
Protein_QC_SC <- as.data.frame(Protein_QC_SC)
Protein_QC_SC <- data.frame(Protein_QC_SC[,-1], row.names = Protein_QC_SC[,1])
row.names(Protein_QC_SC)[1:12] <- paste0(substring(row.names(Protein_QC_SC)[1:12],1,9),substring(row.names(Protein_QC_SC)[1:12],14,16))
row.names(Protein_QC_SC)[13:18] <- paste0(substring(row.names(Protein_QC_SC)[13:18],1,9),substring(row.names(Protein_QC_SC)[13:18],14,16))
row.names(Protein_QC_SC)[19:22] <- paste0(substring(row.names(Protein_QC_SC)[19:22],1,9),substring(row.names(Protein_QC_SC)[19:22],18,20))
row.names(Protein_QC_SC)[23:24] <- paste0(substring(row.names(Protein_QC_SC)[23:24],1,9),substring(row.names(Protein_QC_SC)[23:24],14,16))
row.names(Protein_QC_SC) <- gsub("Stroma","CD45+",rownames(Protein_QC_SC))
Protein_QC_SC <- Protein_QC_SC[order(row.names(Protein_QC_SC)),]
row.names(Protein_QC_SC)[1:12] <- paste0(substr(row.names(Protein_QC_SC)[1:12],1,5),substr(row.names(Protein_QC_SC)[1:12],8,11),
                                              substr(row.names(Protein_QC_SC)[1:12],6,7))
row.names(Protein_QC_SC)[13:24] <- paste0(substr(row.names(Protein_QC_SC)[13:24],1,6),substr(row.names(Protein_QC_SC)[13:24],9,12),
                                               substr(row.names(Protein_QC_SC)[13:24],7,8))
Protein_QC_SC <- Protein_QC_SC[order(row.names(Protein_QC_SC),decreasing=T),]


Processing Gene Expression Data

In [ ]:
#################################################################################################################################################
# EdgeR DEG
#################################################################################################################################################

groups <- factor(substring(row.names(CTA_Bioprobe_QC_SC),1,5))
patients <- factor(rep(c(rep("RO1",4),rep("P04",2),rep("B18",6)),2))

CTA_Bioprobe_QC_SC = as.data.frame(t(CTA_Bioprobe_QC_SC))
CTA_Bioprobe_QC_SC_DEG <- DGEList(CTA_Bioprobe_QC_SC, group=groups, genes= row.names(CTA_Bioprobe_QC_SC))
CTA_Bioprobe_QC_SC_DEG$genes$Symbol <- row.names(CTA_Bioprobe_QC_SC_DEG)
design <- model.matrix(~patients+groups)
rownames(design) <- colnames(CTA_Bioprobe_QC_SC)
keep <- filterByExpr(CTA_Bioprobe_QC_SC_DEG, design)
CTA_Bioprobe_QC_SC_DEG <- CTA_Bioprobe_QC_SC_DEG[keep, , keep.lib.sizes=FALSE]
AveLogCPM <- aveLogCPM(CTA_Bioprobe_QC_SC_DEG)
CTA_Bioprobe_QC_SC_DEG <- calcNormFactors(CTA_Bioprobe_QC_SC_DEG)
boxplot(log(CTA_Bioprobe_QC_SC), xlab="", ylab="Log transformed raw counts",las=2,main="Unnormalised logCPM")

colors <- c("red", "blue", "green")
pch <- c(15,16,17)
plotMDS(CTA_Bioprobe_QC_SC_DEG,col=colors[patients],pch=pch[patients])
legend("center", legend=levels(patients), pch=pch, col=colors, ncol=1)

plotMD(CTA_Bioprobe_QC_SC_DEG, column=1)
abline(h=0, col="red", lty=2, lwd=2)
CTA_Bioprobe_QC_SC_DEG <- estimateDisp(CTA_Bioprobe_QC_SC_DEG, design, robust=TRUE)
plotBCV(CTA_Bioprobe_QC_SC_DEG)

fit <- glmQLFit(CTA_Bioprobe_QC_SC_DEG, design, robust=TRUE)
plotQLDisp(fit)
res_cta <- glmQLFTest(fit, coef=4)
is.de <- decideTestsDGE(res_cta)
plotMD(res_cta, status=is.de)
tr <- glmTreat(fit, coef=4, lfc=log2(1))

logcounts_cta <- edgeR::cpm(CTA_Bioprobe_QC_SC_DEG,log=TRUE)
boxplot(logcounts_cta, xlab="", ylab="Log2 counts per million",las=2,main="Normalised logCPM")
logcounts <- logcounts_cta
logcounts_cta_CD45 <- logcounts_cta[,1:12]
logcounts_cta_PanCK <- logcounts_cta[,13:24]

logcounts_cta_pat <- as.data.frame(logcounts_cta)
my_sample_col <- data.frame(patient = patients)
row.names(my_sample_col) <- colnames(logcounts_cta)
my_sample_col <- as.data.frame(my_sample_col)
my_sample_col$sample <- CTA_Bioprobe_QC_SC_DEG$samples$group
annot_colors=list(sample=c(`CD45+`="#3A54A3",PanCK="#3A783A"),patient=c(B18="#F1A8B3",
                                                                        RO1="#91CFDA",P04="#E96449"))
deg_tr <- as.data.frame(topTags(tr,n=Inf))

Processing Protein Expression Data

In [ ]:
#################################################################################################################################################
# EdgeR Proteins
#################################################################################################################################################

groups_prot <- factor(substring(row.names(Protein_QC_SC),1,5))
patients_prot <- patients
Protein_QC_SC = as.data.frame(t(Protein_QC_SC))
Protein_QC_SC_DEG <- DGEList(Protein_QC_SC, group=groups_prot, genes= row.names(Protein_QC_SC))
Protein_QC_SC_DEG$genes$Symbol <- row.names(Protein_QC_SC_DEG)
design_prot <- model.matrix(~patients_prot+groups_prot)
rownames(design_prot) <- colnames(Protein_QC_SC)
keep_prot <- filterByExpr(Protein_QC_SC_DEG, design_prot)
Protein_QC_SC_DEG <- Protein_QC_SC_DEG[keep_prot, , keep.lib.sizes=FALSE]
AveLogCPM <- aveLogCPM(Protein_QC_SC_DEG)
Protein_QC_SC_DEG <- calcNormFactors(Protein_QC_SC_DEG)
boxplot(log(Protein_QC_SC), xlab="", ylab="Log transformed raw counts",las=2,main="Unnormalised logCPM")

colors <- c("red", "blue", "green")
pch <- c(15,16,17)
plotMDS(Protein_QC_SC_DEG,col=colors[patients_prot],pch=pch[patients_prot])
legend("center", legend=levels(patients_prot), pch=pch, col=colors, ncol=1)

plotMD(Protein_QC_SC_DEG, column=1)
abline(h=0, col="red", lty=2, lwd=2)
Protein_QC_SC_DEG <- estimateDisp(Protein_QC_SC_DEG, design_prot, robust=TRUE)
plotBCV(Protein_QC_SC_DEG)

fit_prot <- glmQLFit(Protein_QC_SC_DEG, design_prot, robust=TRUE)
plotQLDisp(fit_prot)
res_prot <- glmQLFTest(fit_prot, coef=4)
is.de_prot <- decideTestsDGE(res_prot)
plotMD(res_prot, status=is.de_prot)
tr_prot <- glmTreat(fit_prot, coef=4, lfc=log2(1))

logcounts_prot <- edgeR::cpm(Protein_QC_SC_DEG,log=TRUE)
boxplot(logcounts_prot, xlab="", ylab="Log2 counts per million",las=2,main="Normalised logCPM")
logcounts_prot_CD45 <- logcounts_prot[,13:24]
logcounts_prot_PanCK <- logcounts_prot[,1:12]

logcounts_prot_pat <- as.data.frame(logcounts_prot)

de_prot_tr <- as.data.frame(topTags(tr_prot,n=Inf))

## Supplementary Fig10. A

In [ ]:
pheatmap(logcounts_prot_pat[as.data.frame(filter(de_prot_tr,FDR<=0.05 & abs(logFC)>=1))$genes,],
         scale="row",annotation_col = my_sample_col,
         cluster_cols = F, color=colorRampPalette(c("navy", "white", "red"))(50), annotation_colors=annot_colors,
         cellheight = 11, 
         cellwidth = 11, gaps_col=12)

print(paste("The number of Differentially Expressed Proteins from a list of 48 proteins is:",
            length(as.data.frame(filter(de_prot_tr,FDR<=0.05 & abs(logFC)>=1))$genes)))

#write.csv(as.data.frame(filter(de_prot_tr,FDR<=0.05 & abs(logFC)>=1)),"NanoString_GeoMx_DSP_DEPs.csv")
#write.csv(topTags(tr_prot,n=Inf),"NanoString_GeoMx_DSP_EdgeR_proteins_DEGScore.csv")

## Supplementary Fig10. B

In [ ]:
pheatmap(as.matrix(as.data.frame(logcounts_cta)[c("CD163","CD68","MS4A1","CD8A","CD8B","ITGAX","FOXP3","IL2RA","FAP","FN1"),]),
         annotation_col = my_sample_col, scale = "row",fontsize = 10, cellheight = 9, cellwidth = 9,
         cluster_rows=F, cluster_cols = F, gaps_col = c(12), color=colorRampPalette(c("navy", "white", "red"))(100),
         annotation_colors = annot_colors)


## Supplementary Fig10. C

In [ ]:
pheatmap(logcounts_cta_pat[filter(deg_tr,FDR<=0.05 & abs(logFC)>=1.0)$genes[1:30],],
         scale="row",annotation_col = my_sample_col,
         cluster_cols = F, color=colorRampPalette(c("navy", "white", "red"))(50), annotation_colors=annot_colors,
         cellheight = 11, 
         cellwidth = 11, gaps_col=12)


print(paste("The number of Differentially Expressed Proteins from a list of 1820 genes is:",
            length(as.data.frame(filter(deg_tr,FDR<=0.05 & abs(logFC)>=1.0))$genes)))

#write.csv(as.data.frame(filter(deg_tr,FDR<=0.05 & abs(logFC)>=1.0)),"NanoString_GeoMx_DSP_DEGs.csv")
#write.csv(topTags(tr,n=Inf),"NanoString_GeoMx_DSP_EdgeR_genelist_DEGScore.csv")

## Supplementary Fig10. D

In [ ]:
# No proteins overlapped
DE_proteins <- as.data.frame(topTags(res_prot,n=34))
DE_proteins <- DE_proteins[c(all_cancer_PatientOverlapResults$genes,all_normal_PatientOverlapResults$genes),]
DE_proteins <- na.omit(DE_proteins)


DE_genes <- as.data.frame(filter(deg_tr,FDR<=0.05 & abs(logFC)>=1.0))
DE_genes <- DE_genes[c(all_cancer_PatientOverlapResults$genes,all_normal_PatientOverlapResults$genes),]
DE_genes <- na.omit(DE_genes)
DE_genes$group <- c(rep("Cancer",6),rep("Normal",27))
my_sample_row <- DE_genes[c("group")]

pheatmap(as.matrix(as.data.frame(logcounts_cta)[DE_genes$genes,]), 
         annotation_col = my_sample_col, annotation_row = my_sample_row,
         cutree_cols = 2, scale = "row",fontsize = 10, cellheight = 9, cellwidth = 9,
         cluster_rows = FALSE, color=colorRampPalette(c("navy", "white", "red"))(15), annotation_colors=annot_colors,
         cluster_cols = F, gaps_col=12)

## Supplementary Fig10. E

https://github.com/BiomedicalMachineLearning/scc_multiomics/tree/main/Development/SkinCancerAtlasInteractome/FigS4_NanoString/FigS4c

## Supplementary Fig10. F

In [ ]:
#################################################################################################################################################
# Cell De-convolution
#################################################################################################################################################

CTA_Bioprobe_QC_SC_norm <- data.matrix(edgeR::cpm(CTA_Bioprobe_QC_SC,log = TRUE))

per.observation.mean.neg = CTA_Bioprobe_QC_SC_norm["NegProbe", ]
bg = sweep(CTA_Bioprobe_QC_SC_norm * 0, 2, per.observation.mean.neg, "+")
heatmap(sweep(safeTME, 1, apply(safeTME, 1, max), "/"),
        labRow = NA, margins = c(10, 5))

bg2 = derive_GeoMx_background(norm = CTA_Bioprobe_QC_SC_norm,
                              probepool = rep(1, nrow(CTA_Bioprobe_QC_SC_norm)),
                              negnames = "NegProbe")

res = spatialdecon(norm = CTA_Bioprobe_QC_SC_norm,
                   bg = bg,
                   X = safeTME,
                   align_genes = TRUE)

xm <- melt(res$beta, id.vars = "Cell-Type", variable.name="Samples", value.name = "Size")

# Advanced
Segment_Properties <- Segment_Properties[order(Segment_Properties$SegmentLabel),]
Segment_Properties$is_PanCK = (Segment_Properties$SegmentLabel=="PanCK+")
restils = spatialdecon(norm = CTA_Bioprobe_QC_SC_norm,                     
                       raw = data.matrix(CTA_Bioprobe_QC_SC),                        
                       bg = bg, X = safeTME,                     
                       cellmerges = safeTME.matches,   
                       cell_counts = c(864,431,157,137,289,332,66,47,93,227,49,119,330,
                                       416,464,221,516,1040,273,423,352,1165,878,1103),      
                       is_pure_tumor = Segment_Properties$SegmentLabel=="PanCK+",   
                       n_tumor_clusters = 12) 

#################################################################################################################################################
# Cell De-convolution
#################################################################################################################################################

coul3 <- c("deeppink", "gold", "navyblue", "orange", "maroon", "chartreuse4", "darkorchid2",
           "yellowgreen", "darkmagenta", "lightseagreen","gold4", "orangered", "limegreen", "magenta")
order_cell_type <- c("CD4.T.cells", "macrophages", "Treg", "endothelial.cells", "B", "fibroblasts", "NK",
            "CD8.T.cells", "mast", "pDC", "plasma", "monocytes", "mDCs", "neutrophils")

cell_type_proportion <-  restils$prop_of_all 
data_percentage_cell_type_proportion <- apply(cell_type_proportion, 2, function(x){x*100/sum(x,na.rm=T)})
data_percentage_cell_type_proportion_stroma <- data_percentage_cell_type_proportion[,13:24] 
data_percentage_cell_type_proportion_stroma <- data.frame(data_percentage_cell_type_proportion_stroma) %>%slice(match(order_cell_type, 
                                                                                                                    rownames(data_percentage_cell_type_proportion_stroma)))
rownames(data_percentage_cell_type_proportion_stroma) <- c("CD4 T cells", "macrophages", "Treg cells", "endothelial cells", "B cells", "fibroblasts", "NK cells",
                                                          "CD8 T cells", "mast cells", "pDC", "plasma", "monocytes", "mDCs", "neutrophils")
colnames(data_percentage_cell_type_proportion_stroma) <- substr(colnames(data_percentage_cell_type_proportion_stroma),7,22)
bar <- barplot(as.matrix(data_percentage_cell_type_proportion_stroma), col = coul3, border="white", cex.names=1.0, las=2, 
               width=c(330,416,464,221,516,1040,273,423,352,1165,878,1103), space = c(0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5))
legend <- barplot(as.matrix(data_percentage_cell_type_proportion_stroma), col = coul3, legend=TRUE)

#write.csv(data_percentage_cell_type_proportion_stroma,"NanoString_GeoMx_DSP_Cell_type_proportions.csv") 
#save(restils, file="NanoString_GeoMx_DSP_Cell_type_proportions_object.Rdata")